1. 定義讀取圖像 & 3D點座標的函數
2.  讀取相機參數的函數
3.  定義投影點的函數
4.  定義繪製軌跡的函數
5.  定義保存圖像的函數

In [9]:
import numpy as np
import cv2

In [10]:
# 定義相機1的內部參數矩陣
Cam1_K = np.array([[2666.6667, 0., 960.],
                    [0., 2666.6667, 540.],
                    [0., 0., 1.]])

# 定義相機1的外部參數矩陣
Cam1_RT = np.array([[0.9874449, -0.1576334, -0.0102154, 11.840861],
                     [-0.0165432, -0.0388828, -0.9991068, 1.1459197],
                     [0.1570954, 0.986732, -0.0410024, 50.106411]])

# 定義相機2的內部參數矩陣
Cam2_K = np.array([[4266.667, 0., 960.],
                    [0., 4266.667, 540.],
                    [0., 0., 1.]])

# 定義相機2的外部參數矩陣
Cam2_RT = np.array([[-0.0000004, -1., 0.0000001, 12.915463],
                     [-0.299365, 0.0, -0.9541386, -7.8007503],
                     [0.9541386, -0.0000004, -0.299365, 116.06975]])

In [11]:
def read_images(image1_path, image2_path):
    # 讀取圖像
    image1 = cv2.imread(image1_path)
    image2 = cv2.imread(image2_path)
    return image1, image2
def read_points(points_file):
    # 讀取3D點座標
    points_3d = np.loadtxt(points_file)
    return points_3d

In [12]:
def project_points(Cam_K, Cam_RT, points_3d, image_size):

    ## 計算相機的投影矩陣
    proj_matrix = np.dot(Cam_K, Cam_RT)
    # print('proj_matrix.shape',proj_matrix.shape)
    # print('points_3d.shape',points_3d.shape)
    

    ## 將三維點擴展為齊次座標
    points_3d = np.append(points_3d, 1)
    points_3d_h = points_3d.reshape(-1, 1) # Reshape the point to a column vector
    # print('points_3d_h.shape',points_3d_h.shape)
    
    
    ## 將三維點投影到圖像平面
    points_2d_h = np.dot(proj_matrix, points_3d_h)
    points_2d = points_2d_h[:2, :] / points_2d_h[2, :]
    # print('result',points_2d.shape)


    ## 將超出圖像範圍的點移除
    points_2d = points_2d[:, (points_2d[0] >= 0) & (points_2d[0] < image_size[1]) & 
                              (points_2d[1] >= 0) & (points_2d[1] < image_size[0])]

    return points_2d

In [13]:
def draw_and_save_trajectory(image, projected_points, output_path, student_id):
    # 繪製投影點
    for point in projected_points.T:  # 轉置投影點的矩陣以便遍歷
        x, y = int(point[0]), int(point[1])
        cv2.circle(image, (x, y), 5, (0, 255, 0), -1)  # 在圖像上畫一個綠色的點
    
    # 在左下角繪製名字和學號
    font = cv2.FONT_HERSHEY_SIMPLEX
    bottom_left_corner = (50, image.shape[0] - 50)  # 左下角位置
    font_scale = 1
    font_color = (255, 255, 255)  # 白色字體
    font_thickness = 2
    cv2.putText(image, f'{student_id}', bottom_left_corner, font, font_scale, font_color, font_thickness)

    # 保存圖像
    cv2.imwrite(output_path, image)

In [14]:
# 設定檔案路徑
image1_path = "./SceneFromCamera1.jpg"
image2_path = "SceneFromCamera2.jpg"
points_file = "3D_Trajectory.xyz"
camera_parameter_file = "CameraParameter.txt"
output_image1 = "M11202130_1.jpg"
output_image2 = "M11202130_2.jpg"

In [15]:
image1, image2 = read_images(image1_path, image2_path)
points_3d = read_points(points_file)

In [16]:
image_size = (1080, 1920)  # 圖像大小
img_1_result = np.empty((2, 0))
img_2_result = np.empty((2, 0))
for point in points_3d:
    img_1_projected_point = project_points(Cam1_K, Cam1_RT, point, image_size)
    img_2_projected_point = project_points(Cam2_K, Cam2_RT, point, image_size)
    # print(projected_point.shape)
    img_1_result = np.append(img_1_result, img_1_projected_point, axis=1)
    img_2_result = np.append(img_2_result, img_2_projected_point, axis=1)

In [17]:
# 調用函數繪製投影點並保存圖像
student_id = "M11202130" 
draw_and_save_trajectory(image1, img_1_result, output_image1, student_id)
draw_and_save_trajectory(image2, img_2_result, output_image2, student_id)
# cv2.imshow('result',image )
# cv2.waitKey()
# cv2.destroyAllWindows()